# Weather lookup helper
Use the free Open-Meteo API: geocode the city name, then fetch current weather. No API key required.

In [3]:
import requests

GEOCODE_URL = "https://geocoding-api.open-meteo.com/v1/search"
FORECAST_URL = "https://api.open-meteo.com/v1/forecast"


def get_weather_by_city(city: str) -> dict:
    """Return current weather for a city using Open-Meteo.

    Raises ValueError for missing city or if the city cannot be resolved.
    Raises requests.RequestException for network-related issues.
    """
    if not city or not city.strip():
        raise ValueError("City name must be provided")

    # 1) Geocode the city name -> latitude/longitude
    params = {"name": city.strip(), "count": 1, "language": "en"}
    geo_resp = requests.get(GEOCODE_URL, params=params, timeout=10)
    geo_resp.raise_for_status()
    geo = geo_resp.json()

    if not geo.get("results"):
        raise ValueError(f"No location found for '{city}'")

    first = geo["results"][0]
    lat, lon = first["latitude"], first["longitude"]

    # 2) Fetch current weather for that lat/lon
    forecast_params = {
        "latitude": lat,
        "longitude": lon,
        "current_weather": True,
        "timezone": "auto",
    }
    forecast_resp = requests.get(FORECAST_URL, params=forecast_params, timeout=10)
    forecast_resp.raise_for_status()
    forecast = forecast_resp.json()

    current = forecast.get("current_weather") or {}
    return {
        "city": first.get("name"),
        "country": first.get("country"),
        "latitude": lat,
        "longitude": lon,
        "temperature_c": current.get("temperature"),
        "windspeed_kmh": current.get("windspeed"),
        "winddirection_deg": current.get("winddirection"),
        "timestamp": current.get("time"),
        "raw": current,
    }


In [4]:
# Example usage
try:
    data = get_weather_by_city("London")
    print(data)
except Exception as exc:
    print(f"Error: {exc}")

{'city': 'London', 'country': 'United Kingdom', 'latitude': 51.50853, 'longitude': -0.12574, 'temperature_c': 7.7, 'windspeed_kmh': 7.6, 'winddirection_deg': 161, 'timestamp': '2026-01-14T14:45', 'raw': {'time': '2026-01-14T14:45', 'interval': 900, 'temperature': 7.7, 'windspeed': 7.6, 'winddirection': 161, 'is_day': 1, 'weathercode': 2}}


In [7]:
import os
import json
from openai import OpenAI

# LLM client (Gemini via OpenAI-compatible API, same pattern as app.py)
_google_api_key = os.getenv("GOOGLE_API_KEY")
if not _google_api_key:
    raise ValueError("GOOGLE_API_KEY is not set. Add it to your .env or environment.")

weather_llm = OpenAI(
    api_key=_google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# Tool schema exposed to the model
get_weather_by_city_json = {
    "name": "get_weather_by_city",
    "description": "Get the current weather for a given city name.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city the tourist is visiting (e.g. 'London').",
            },
        },
        "required": ["city"],
        "additionalProperties": False,
    },
}

weather_tools = [{"type": "function", "function": get_weather_by_city_json}]


def _handle_weather_tool_calls(tool_calls):
    """Execute tool calls from the LLM and return tool messages."""
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        if tool_name == "get_weather_by_city":
            result = get_weather_by_city(**arguments)
        else:
            result = {"error": f"Unknown tool {tool_name}"}
        results.append(
            {
                "role": "tool",
                "content": json.dumps(result),
                "tool_call_id": tool_call.id,
            }
        )
    return results


def _weather_system_prompt():
    return (
        "You are a helpful travel assistant. "
        "Use the get_weather_by_city tool to look up the current weather for the city the tourist is visiting, "
        "then advise clearly whether they should pack mostly WINTER clothing (coats, sweaters, warm layers) "
        "or SUMMER clothing (shorts, t‑shirts, light layers). "
        "\n\nGuidelines based on temperature in Celsius: "
        "\n- <= 10°C: Definitely WINTER clothing."
        "\n- 10–18°C: Mostly cool‑weather layers (lean winter)."
        "\n- 18–24°C: Mixed but lean SUMMER."
        "\n- > 24°C: Definitely SUMMER clothing."
        "\nAlways mention the approximate temperature and a short packing list."
    )


def tourist_weather_chat(message, history=None):
    """LLM chat that uses the weather tool to give packing advice.

    Parameters
    ----------
    message : str
        Latest user message.
    history : list[dict] | None
        Previous chat history in OpenAI/gradio "messages" format,
        e.g. [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}].
    """
    if history is None:
        history = []

    messages = [
        {"role": "system", "content": _weather_system_prompt()},
        *history,
        {"role": "user", "content": message},
    ]

    print("\n" + "="*40)
    print("User message:")
    print(f"{message}")
    print("="*40 + "\n")

    done = False
    while not done:
        response = weather_llm.chat.completions.create(
            model="gemini-2.0-flash",
            messages=messages,
            tools=weather_tools,
        )
        choice = response.choices[0]
        if choice.finish_reason == "tool_calls":
            tool_message = choice.message
            print(f"\ntool_message={tool_message}")
            tool_calls = tool_message.tool_calls
            tool_results = _handle_weather_tool_calls(tool_calls)
            print(f"\ntool_results={tool_results}")
            messages.append(tool_message)
            messages.extend(tool_results)
        else:
            done = True
            print("\n no further tool calls required")            
            print(f"\nchoice.message.content={choice.message.content}")            
            return choice.message.content


In [ ]:
import gradio as gr
gr.ChatInterface(tourist_weather_chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.



User message:
I am visiting rome


tool_message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='function-call-18306298596645819880', function=Function(arguments='{"city":"Rome"}', name='get_weather_by_city'), type='function')])

tool_results=[{'role': 'tool', 'content': '{"city": "Rome", "country": "Italy", "latitude": 41.89193, "longitude": 12.51133, "temperature_c": 14.7, "windspeed_kmh": 9.0, "winddirection_deg": 151, "timestamp": "2026-01-14T16:00", "raw": {"time": "2026-01-14T16:00", "interval": 900, "temperature": 14.7, "windspeed": 9.0, "winddirection": 151, "is_day": 1, "weathercode": 2}}', 'tool_call_id': 'function-call-18306298596645819880'}]

 no further tool calls required

choice.message.content=The current temperature in Rome is approximately 14.7°C. I would advise you to pack mostly cool-weather layers. Consider bringing sweaters, light jackets, an